In [21]:
import pandas as pd
import numpy as np

from skopt.space import Real, Integer
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.optimizers import Adam

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

Neural Network Model for Hyper Parameter Tuning

In [22]:
def branin(X, Y):
    a = 1.0
    b = 5.1 / (4.0 * np.pi**2)
    c = 5.0 / np.pi
    r = 6.0
    s = 10.0
    t = 1.0 / (8.0 * np.pi)
    
    return a * (Y - b * X**2 + c * X - r)**2 + s * (1 - t) * np.cos(X) + s

def currin(X, Y):
    term1 = 1 - np.exp(-1 / (2 * Y))
    term2 = (2300 * X**3 + 1900 * X**2 + 2092 * X + 60) / (100 * X**3 + 500 * X**2 + 4 * X + 20)
    
    return term1 * term2

In [23]:
# Random Noise that will be added to data Response
ran_err = lambda n, x: np.random.normal(0, x, n)

In [24]:
# Generate 10 random data points for two inputs (X, Y) within specified bounds
X = np.random.uniform(-5, 10, 10)
Y = np.random.uniform(0, 15, 10)

# Calculate the outputs using Branin and Currin functions
output1 = branin(X, Y)
output2 = currin(X, Y)

# Generate Initial Dataset
df = pd.DataFrame({
    'x': X,
    'y': Y,
    'z1': output1,
    'z2': output2
})


In [25]:
# Define the hyperparameter space
hyp_space  = [
    Integer(2, 10, name='num_layers'), # Num of layers in the network (depth)
    Integer(50, 200, name='num_units'), # Num of neurons in each hidden layer (width)
    Real(0.00001, 0.1, prior='log-uniform', name='learning_rate'), # Steps size at each iteration 
    Real(0.0, 0.5, name='dropout_rate'), # Probability of droping out a neuron
    Integer(20, 150, name='batch_size'), # Num of samples per batch
    Integer(50, 200, name='epochs') # Num of epochs (iterations over the entire dataset) during training
]

# Defining the Neural Network Model
def NN_model(num_layers, num_units, learning_rate, dropout_rate):
    inputs = Input(shape=(2,))
    x = Dense(num_units, activation='relu', kernel_regularizer='l2')(inputs)
    for _ in range(num_layers - 1):
        x = Dense(num_units, activation='relu', kernel_regularizer='l2')(x)
        x = Dropout(dropout_rate)(x)
    outputs = Dense(2, activation='linear')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.problem import Problem
from pymoo.termination import get_termination
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.crossover.sbx import SimulatedBinaryCrossover
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.optimize import minimize

# Define the objective functions based on the neural network outputs
# Problem is initialized with 2 decision vars and two objectives
class NNProblem(Problem):
    def __init__(self, model, scaler, maximize):
        super().__init__(n_var=2, n_obj=2, n_constr=0, xl=np.array([-15, 0]), xu=np.array([10, 15])) 
        self.model = model
        self.scaler = scaler
        self.maximize = maximize

    # Input data is scaled using a pre-fitted sacler
    # Predictions are made using NN model and stores them in an array
    def _evaluate(self, X, out, *args, **kwargs):
        X_scaled = self.scaler.transform(X)
        predictions = self.model.predict(X_scaled)
        # Adjusting objectives based on whether they should be maximized or minimized
        for i in range(predictions.shape[1]):
            if self.maximize[i]:
                predictions[:, i] = -predictions[:, i]  # Convert maximization to minimization
        out["F"] = predictions

In [26]:
def perform_iteration(df):
    # Define the objective function to minimize
    @use_named_args(hyp_space)
    def objective(**params):
        num_layers = params['num_layers']
        num_units = params['num_units']
        learning_rate = params['learning_rate']
        dropout_rate = params['dropout_rate']
        batch_size = params['batch_size']
        epochs = params['epochs']

        model = NN_model(num_layers, num_units, learning_rate, dropout_rate)
    
        # Define K-fold cross-validation
        kfold = KFold(n_splits=3)
        scores = []

        # Perform cross-validation
        for train_idx, val_idx in kfold.split(df[['x', 'y']]):
            X_train, X_val = df[['x', 'y']].iloc[train_idx], df[['x', 'y']].iloc[val_idx]
            y_train, y_val = df[['z1', 'z2']].iloc[train_idx], df[['z1', 'z2']].iloc[val_idx]

            # Standardize the features
            scaler_x = StandardScaler().fit(X_train)
            X_train_scaled = scaler_x.transform(X_train)
            X_val_scaled = scaler_x.transform(X_val)

            # Train the model
            model.fit(X_train_scaled, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

            # Evaluate the model
            score = model.evaluate(X_val_scaled, y_val, verbose=0)
            scores.append(score)
    
        return np.mean(scores)

    # Perform Bayesian optimization
    result = gp_minimize(objective, hyp_space, n_calls=50, random_state=0, acq_func='EI')

    # Output best hyperparameters from BayesOpt
    print("Best hyperparameters:")
    print("num_layers:", result.x[0])
    print("num_units:", result.x[1])
    print("learning_rate:", result.x[2])
    print("dropout_rate:", result.x[3])
    print("batch_size:", result.x[4])
    print("epochs:", result.x[5])

    # Train the model with the best hyperparameters on the entire dataset
    best_model = NN_model(
        num_layers=result.x[0],
        num_units=result.x[1],
        learning_rate=result.x[2],
        dropout_rate=result.x[3]
    )

    # Standardize the initial dataset
    scaler_x = StandardScaler().fit(df[['x', 'y']])
    X_scaled = scaler_x.transform(df[['x', 'y']])

    # Train the best model on the initial dataset
    best_model.fit(X_scaled, df[['z1', 'z2']], epochs=result.x[5], batch_size=result.x[4], verbose=1)

    # Define the problem for NSGA-II
    maximize = [False, False]
    problem = NNProblem(best_model, scaler_x, maximize)

    # Defining the NSGA-II algorithm
    algorithm = NSGA2(
        pop_size=10,
        sampling=FloatRandomSampling(),
        crossover=SimulatedBinaryCrossover(prob=0.9, eta=15),
        mutation=PolynomialMutation(eta=20),
        eliminate_duplicates=True
    )

    # Perform the optimization
    termination = get_termination("n_gen", 100) # Sets the termination criterion to stop after 100 generations
    res = minimize(problem, algorithm, termination, seed=1, save_history=True, verbose=True)

    # Extract the Pareto-optimal solutions
    pareto_set = res.X
    pareto_front = res.F

    # Choose the next samples based on Pareto-optimal solutions
    next_samples = pd.DataFrame(pareto_set, columns=['x', 'y'])
    next_samples["z1"] = pareto_front[:, 0]
    next_samples["z2"] = pareto_front[:, 1]

    # Extract x and y columns from next_samples DataFrame
    X_new = next_samples[['x', 'y']].values

    # Scale the extracted data
    X_new_scaled = scaler_x.transform(X_new)
    actual_values = branin(next_samples['x'], next_samples['y'])
    actual_values2 = currin(next_samples['x'], next_samples['y'])

    validated_values = pd.DataFrame(actual_values, columns=['z1'])
    validated_values["z2"] = actual_values2

    validated_values = pd.concat([next_samples[['x', 'y']], validated_values], axis=1)
    validated_values

    # Validate prposed values with simulation
    loss = best_model.evaluate(X_new_scaled, validated_values[['z1', 'z2']])

    loss_values = []

    # Store the loss value in the list
    loss_values.append(loss)

    # Convert the list to a DataFrame
    loss_df = pd.DataFrame(loss_values, columns=['loss'])

    # Print the DataFrame
    print("Loss values stored in a DataFrame:")
    print(loss_df)

    # Add validated data points to original data frame
    df = pd.concat([df, validated_values], axis=0, ignore_index=True)
    return df


In [28]:
df = perform_iteration(df)

Best hyperparameters:
num_layers: 3
num_units: 145
learning_rate: 0.03177648779036605
dropout_rate: 0.5
batch_size: 150
epochs: 117
Epoch 1/117
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 1512.0525
Epoch 2/117
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1333.2566
Epoch 3/117
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 737.9782
Epoch 4/117
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 448.2868
Epoch 5/117
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 508.0007
Epoch 6/117
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 114.3445
Epoch 7/117
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 269.4361
Epoch 8/117
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 145.4869
Epoch 9/117
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 90.7068
Epoch 10/117
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 198.5102
Epoch 11/117
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 336.4151
Epoch 12/117
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 166.3128
Epoch 13/117
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 315

c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
     2 |       20 |     10 |  0.0362172187 |             f
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
     3 |       30 |     10 |  0.0234478180 |         ideal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


     4 |       40 |     10 |  0.0296856092 |         ideal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
     5 |       50 |     10 |  0.0162158546 |             f
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


...

c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


    98 |      980 |     10 |  0.0139051756 |             f
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
    99 |      990 |     10 |  0.0216790884 |             f
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


   100 |     1000 |     10 |  0.0179001289 |             f
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - loss: 1497.5917
Loss values stored in a DataFrame:
          loss
0  1497.591675


c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [29]:
df = perform_iteration(df)

Best hyperparameters:
num_layers: 6
num_units: 50
learning_rate: 0.0033636752977087387
dropout_rate: 0.0
batch_size: 20
epochs: 200
Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 4530.1802
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 4524.6714
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 4519.4985
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 4513.8628
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 4506.4292
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 4496.3960
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 4483.0376
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 4465.7891
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 4443.3838
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 4413.8452
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 4375.2905
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 4325.0781
Epoch 13/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step 

c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
     2 |       20 |      3 |  1.2123288533 |         ideal


c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
     3 |       30 |      4 |  0.0332015875 |         ideal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
     4 |       40 |      5 |  0.3394203150 |         nadir


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
    97 |      970 |     10 |  0.0070473457 |             f
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
    98 |      980 |     10 |  0.0095928236 |             f


c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
    99 |      990 |     10 |  0.0086519631 |             f
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
   100 |     1000 |     10 |  0.0218263717 |             f


c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 19.2519
Loss values stored in a DataFrame:
        loss
0  19.251862


In [30]:
df

,x,y,z1,z2
0,3.882433,1.238570e+00,3.195042,4.241346
1,-1.263980,9.835033e+00,15.514625,-0.339727
2,8.634894,1.095936e+01,85.506584,0.719315
3,8.554015,9.808257e+00,67.331818,0.799645
4,4.355560,9.806835e+00,75.339826,0.657504
5,2.244323,3.239995e+00,4.036714,1.567934
6,6.593115,1.033280e+01,103.978458,0.707633
7,5.954446,2.335695e+00,20.606233,2.802407
8,-1.249144,1.212510e+01,28.523347,-0.274010
9,0.560863,4.200307e+00,19.029146,1.273153


In [31]:
df = perform_iteration(df)

Best hyperparameters:
num_layers: 4
num_units: 50
learning_rate: 0.1
dropout_rate: 0.0
batch_size: 20
epochs: 200
Epoch 1/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2920.6270
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1349.3102 
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2225.3838 
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 922.9809 
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 433.7600 
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 597.8166 
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 201.1163 
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 158.2903 
Epoch 9/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 344.5016 
Epoch 10/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 193.6522 
Epoch 11/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 115.5328 
Epoch 12/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 184.5484 
Epoch 13/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 142.2662
Epoch 14/2

c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
     2 |       20 |      4 |  0.000000E+00 |             f
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
     3 |       30 |     10 |  0.000000E+00 |             f
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
     4 |       40 |     10 |  0.000000E+00 |             f
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
     5 |       50 |     10 |  0.000000E+00 |             f
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

...

    97 |      970 |     10 |  0.000000E+00 |             f
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
    98 |      980 |     10 |  0.000000E+00 |             f
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


    99 |      990 |     10 |  0.000000E+00 |             f
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
   100 |     1000 |     10 |  0.000000E+00 |             f


c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Simon\anaconda3\envs\research\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 218156.2500
Loss values stored in a DataFrame:
        loss
0  218156.25


In [32]:
df

,x,y,z1,z2
0,3.882433,1.238570e+00,3.195042,4.241346
1,-1.263980,9.835033e+00,15.514625,-0.339727
2,8.634894,1.095936e+01,85.506584,0.719315
3,8.554015,9.808257e+00,67.331818,0.799645
4,4.355560,9.806835e+00,75.339826,0.657504
5,2.244323,3.239995e+00,4.036714,1.567934
6,6.593115,1.033280e+01,103.978458,0.707633
7,5.954446,2.335695e+00,20.606233,2.802407
8,-1.249144,1.212510e+01,28.523347,-0.274010
9,0.560863,4.200307e+00,19.029146,1.273153
